## Importing Spacy

SpaCy is a library that allows you to perform many NLP tasks, including preprocessing text by exploiting not only a rule based approach, but also some pretrained models, for example to split a text into sentences, or find the named entities contained in the text.

To take advantage of it, you need to download it as explained in the following cell. For english, a common choice is `en_core_web_sm`, but also `en_core_web_lg` results in a good performance while being lightweight.

In [1]:
# !conda install -c conda-forge spacy
# !python -m spacy download en_core_web_sm
## I recommend the one above, because the following is more accurate but less efficient
# !python -m spacy download en_core_web_lg

In [2]:
import spacy


nlp = spacy.load("en_core_web_sm")
# You can also load en_core_web_lg that has an higher accuracy but it's less efficient
# nlp = spacy.load("en_core_web_lg")



By running `nlp.pipeline` you can see what are the steps that spaCy automatically does for you. 

In [3]:
print(nlp.pipeline)

[('tagger', <spacy.pipeline.pipes.Tagger object at 0x7fda6071ef10>), ('parser', <spacy.pipeline.pipes.DependencyParser object at 0x7fda5064b1c0>), ('ner', <spacy.pipeline.pipes.EntityRecognizer object at 0x7fda5064b100>)]


To use it, if you have loaded the model in a variable named `nlp` as above (`nlp = spacy.load("en_core_web_sm")`), you can just pass the text you want as argument as follows:

In [4]:
# Process sentences 'Hello, world. Antonio is learning Python.' using spaCy
doc = nlp(u"Hello, world. Antonio is learning Python.")

By doing so, a lot of things happened! You can iterate over the "doc" and you will get the tokens of the text:

In [5]:
for token in doc:
    print(token.text)

Hello
,
world
.
Antonio
is
learning
Python
.


In [6]:
# Get first token of the processed document
token = doc[0]
print(token)

Hello


The spaCy model automatically divide the text in sentences:

In [7]:
# Print sentences (one sentence per line)
for sent in doc.sents:
    print(sent)

Hello, world.
Antonio is learning Python.


It could look a trivial task, like it was a fancy way of doing `text.split(".")`. However, how would you split into sentences the text "Im antonio im learning python"?

We know that "im" is a wrong way to write "I'm" and, since there are two verbs in that text, there are also two sentences. Let's see how spaCy performs:

In [8]:
doc = nlp("Im antonio im learning python")
for sentence in doc.sents:
    print(sentence)

Im antonio
im learning python


Nice! However, don't get used to this, because there are tons of more complicated sentences that can easily be misinterpreted.

In the example below, you can see that spaCy is smart enough to consider N.Y. as a single token, and not as two:

In [9]:
tokens = nlp("Let's go to N.Y.!")

In [10]:
for token in tokens:
    print(token.text)

Let
's
go
to
N.Y.
!


As you have seen, using `nlp`, that comes from `spacy.load("en_core_web_sm")`, you get the tokenized version of the sentence. If you want only the instance of the `Tokenizer` class, you can run:

In [9]:
tokenizer = nlp.tokenizer
type(tokenizer)

spacy.tokenizer.Tokenizer

<IPython.core.display.Javascript object>

If you want to instantiate a custom one, with rules and prefixes and so on:

In [10]:
from spacy.tokenizer import Tokenizer

tokenizer = Tokenizer(vocab=nlp.vocab)

<IPython.core.display.Javascript object>

The tokenizer defined above contains only english rules.
Let's test it on "Let's go to N.Y.!"

In [11]:
tokens = tokenizer("Let's go to N.Y.!")
for token in tokens:
    print(token)



Let's
go
to
N.Y.!


<IPython.core.display.Javascript object>

As you can see here, it doesn't handle the exceptions about the dots. 

Looking at the output of `nlp.pipeline` above, we can see there are a tagger, a dependency parser and the entity recognizer. Let's check the entities of the following sentence:

In [14]:
doc = nlp("Apple is a $1000b company.")


In [15]:
for token in doc:
    print(token)

Apple
is
a
$
1000b
company
.


In [24]:
for ent in doc.ents:
    print(ent, ent.label_)

Apple ORG
1000b MONEY


<IPython.core.display.Javascript object>

## Removing stop words

In general, it's convenient to remove all the stop words, *i.e. very common words in a language*, because they don't help most of NLP problem such as semantic analysis.

In [35]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
print("Number of stop words: %d" % len(spacy_stopwords))
print("First ten stop words: %s" % list(spacy_stopwords)[:10])

Number of stop words: 326
First ten stop words: ['front', 'within', 'down', 'besides', 'each', 'until', 'more', 'take', 'have', 'various']


<IPython.core.display.Javascript object>

To remove them:

In [36]:
text = """He determined to drop his litigation with the monastry, and relinguish his claims to the wood-cuting and 
fishery rihgts at once. He was the more ready to do this becuase the rights had become much less valuable, and he had 
indeed the vaguest idea where the wood and river in question were."""

doc = nlp(text)

tokens = [token.text for token in doc if not token.is_stop]
for token in tokens:
    print(token)

determined
drop
his litigation
the monastry
,
relinguish
his claims
wood
-
cuting


fishery rihgts
.
ready
this becuase
the rights
valuable
,


indeed the vaguest idea
the wood
river
question
.


<IPython.core.display.Javascript object>

For adding customized stop words:

In [39]:
customize_stop_words = ["computing", "filtered"]
for w in customize_stop_words:
    nlp.vocab[w].is_stop = True

<IPython.core.display.Javascript object>

## Stemming and Lemmatization

In most natural languages, a root word can have many variants. For example, the word ‘play’ can be used as ‘playing’, ‘played’, ‘plays’, etc. You can think of similar examples (and there are plenty).

**Stemming**

Let’s first understand stemming:

Stemming is a text normalization technique that cuts off the end or beginning of a word by taking into account a list of common prefixes or suffixes that could be found in that word
It is a rudimentary rule-based process of stripping the suffixes (“ing”, “ly”, “es”, “s” etc) from a word
 

**Lemmatization**

Lemmatization, on the other hand, is an organized & step-by-step procedure of obtaining the root form of the word. It makes use of vocabulary (dictionary importance of words) and morphological analysis (word structure and grammar relations).

Stemming algorithm works by cutting the suffix or prefix from the word. Lemmatization is a more powerful operation as it takes into consideration the morphological analysis of the word.

Lemmatization returns the lemma, which is the root word of all its inflection forms.

We can say that stemming is a quick and dirty method of chopping off words to its root form while on the other hand, lemmatization is an intelligent operation that uses dictionaries which are created by in-depth linguistic knowledge. Hence, Lemmatization helps in forming better features.

In [40]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(nlp.create_pipe("merge_entities"))
# not using merge_chunk_nouns
doc = nlp(
    u"""He determined to drop his litigation with the monastry, and relinguish his claims to the wood-cuting and 
fishery rihgts at once. He was the more ready to do this becuase the rights had become much less valuable, and he had 
indeed the vaguest idea where the wood and river in question were."""
)

lemma_word1 = []
for token in doc:
    if token.is_stop:
        continue
    lemma_word1.append(token.lemma_)
lemma_word1

['determine',
 'drop',
 'litigation',
 'monastry',
 ',',
 'relinguish',
 'claim',
 'wood',
 '-',
 'cut',
 '\n',
 'fishery',
 'rihgts',
 '.',
 'ready',
 'becuase',
 'right',
 'valuable',
 ',',
 '\n',
 'vague',
 'idea',
 'wood',
 'river',
 'question',
 '.']

<IPython.core.display.Javascript object>

## Removing the punctuation



In [41]:
text = """He determined to drop his litigation with the monastry, and relinguish his claims to the wood-cuting and 
fishery rihgts at once. He was the more ready to do this becuase the rights had become much less valuable, and he had 
indeed the vaguest idea where the wood and river in question were."""


import string

text_no_punct = "".join([char for char in text if char not in string.punctuation])

text_no_punct

'He determined to drop his litigation with the monastry and relinguish his claims to the woodcuting and \nfishery rihgts at once He was the more ready to do this becuase the rights had become much less valuable and he had \nindeed the vaguest idea where the wood and river in question were'

<IPython.core.display.Javascript object>

In [42]:
doc = nlp(text_no_punct)
for token in doc:
    print(token)

He
determined
to
drop
his
litigation
with
the
monastry
and
relinguish
his
claims
to
the
woodcuting
and


fishery
rihgts
at
once
He
was
the
more
ready
to
do
this
becuase
the
rights
had
become
much
less
valuable
and
he
had


indeed
the
vaguest
idea
where
the
wood
and
river
in
question
were


<IPython.core.display.Javascript object>

For text extracted from dialogues or chats, it is convenient to preprocess the text so that multiple occurrences of the same characters get condensed into one or two, and then use a spell checker to find the correct form of the word.

A way to do that is to replace all the occurrences of repeated characters with a single one and then use a spell checker: "hhheeelllllooo hoooowww areee youuu?" becomes "helo how are you?" and then the spell checker would make it "hello how are you?"




In [99]:
st = "hhheeeLLLLooo hoooowww areee youuu?????"
text = re.sub(r"(.)\1+", r"\1", st)
text

'heLo how are you?'

<IPython.core.display.Javascript object>

In [100]:
from spellchecker import SpellChecker

text = nlp(text)
spell = SpellChecker()

# find those words that may be misspelled
misspelled = spell.unknown([token.text for token in text])


for word in misspelled:
    # Get the one `most likely` answer
    print(spell.correction(word))

    # Get a list of `likely` options
    print(spell.candidates(word))

<IPython.core.display.Javascript object>

It didn't find any mispelled (even if there was "helo"). Try another spell checker:

https://github.com/fsondej/autocorrect

In [95]:
from autocorrect import Speller

spell = Speller()

spell(text.text)

'hero how are you?'

<IPython.core.display.Javascript object>

As you can see, it's not always working properly! However, overall it should improve your text.

If you want to create a separate lemmatizer instead of having it in the pipeline:

**For spacy before v3**

In [4]:
from spacy.lemmatizer import Lemmatizer, ADJ, NOUN, VERB

lemmatizer = nlp.vocab.morphology.lemmatizer
print(lemmatizer("studying", VERB))
print(lemmatizer("studying", NOUN))
print(lemmatizer("studying", ADJ))

# or as alternative

print(lemmatizer.verb("studying"))
print(lemmatizer.noun("studying"))
print(lemmatizer.adj("studying"))

['study']
['studying']
['studying']
['study']
['studying']
['studying']


spaCy has no built-in stemming! However, Lemmatization is enough for most of the tasks. As alternative, you can use [NLTK library](https://www.nltk.org).

## Part of Speech (POS) Tagging

Parts of speech tagging simply refers to assigning parts of speech to individual words in a sentence, which means that, unlike phrase matching, which is performed at the sentence or multi-word level, parts of speech tagging is performed at the token level.

In [7]:
sentence = nlp("Antonio is learning Python in Strive School.")

for token in sentence:
    print(token.pos_)

PROPN
AUX
VERB
PROPN
ADP
PROPN
PROPN
PUNCT


The `.pos_` attribute gives the *coarse-grained* POS tag. To inspect the *fine-grained* POS tags we could use the `.tag_`attribute:

In [8]:
sentence = nlp("Antonio is learning Python in Strive School.")

for token in sentence:
    print(token.tag_)

NNP
VBZ
VBG
NNP
IN
NNP
NNP
.


While the output of the `.pos_` attribute is easy to decrypt (`PROPN`: proper noun,
`AUX`: Auxiliary verb,
`VERB`: verb,
`ADP`: Adposition,
`PUNCT`: Punctuation), the `.tag_`'s output is more cryptic. For this, you can use the `spacy.explain()` function to get the intuition behind that:


In [9]:
for token in sentence:
    print(spacy.explain(token.tag_))

noun, proper singular
verb, 3rd person singular present
verb, gerund or present participle
noun, proper singular
conjunction, subordinating or preposition
noun, proper singular
noun, proper singular
punctuation mark, sentence closer


Go and dig up your primary school grammar book!

Let's put everything together:

In [10]:
for token in sentence:
    print(f'{token.text:{12}} {token.pos_:{10}} {token.tag_:{8}} {spacy.explain(token.tag_)}')


Antonio      PROPN      NNP      noun, proper singular
is           AUX        VBZ      verb, 3rd person singular present
learning     VERB       VBG      verb, gerund or present participle
Python       PROPN      NNP      noun, proper singular
in           ADP        IN       conjunction, subordinating or preposition
Strive       PROPN      NNP      noun, proper singular
School       PROPN      NNP      noun, proper singular
.            PUNCT      .        punctuation mark, sentence closer


(the numbers between curly brackets define spaces for a better formatting).

You can count the number of occurrences of each POS tag by calling the `count_by` method. 

The syntax is as follows (you need to pass `spacy.attrs.POS` as argument of the method):

In [11]:
sentence = nlp("Antonio is learning Python Programming Language")

num_pos = sentence.count_by(spacy.attrs.POS)
num_pos

{96: 4, 87: 1, 100: 1}

The keys of the vocabulary are the ID of the POS tags, the values are their frequencies of occurrence. To retrieve the POS tags given the ID, you can do as follows:

In [19]:
sentence.vocab[96].text

'PROPN'

where 96 is the ID of the tag. Printing all together:

In [18]:
for ID, frequency in num_pos.items():
    print(f"{ID} stands for {sentence.vocab[ID].text:{8}}: {frequency}")

96 stands for PROPN   : 4
87 stands for AUX     : 1
100 stands for VERB    : 1


## Named Entity Recognition

A named entity is a “real-world object” that’s assigned a name – for example, a person, a country, a product or a book title. spaCy can recognize various types of named entities in a document, by asking the model for a prediction. Because models are statistical and strongly depend on the examples they were trained on, this doesn’t always work perfectly and might need some tuning later, depending on your use case.

Named entities are available as the ents property of a Doc.


Example:



In [129]:
doc = nlp("Antonio works at Strive School.")

<IPython.core.display.Javascript object>

In [131]:
from spacy import displacy

displacy.render(doc, style="ent")

<IPython.core.display.Javascript object>

In [136]:
doc = nlp("Rome is a big city.")

<IPython.core.display.Javascript object>

In [137]:
displacy.render(doc, style="ent")

<IPython.core.display.Javascript object>

ORG stands for organization, GPE stands for Geopolitical Entity. Some other tags are:

In spaCy you can list the entities by doing:
    

In [25]:
doc = nlp('Manchester United is looking to sign Harry Kane for $90 million')

In [26]:
doc.ents

(Manchester United, Harry Kane, $90 million)

We can access the entities text, label by doing:

In [27]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Manchester United ORG
Harry Kane PERSON
$90 million MONEY


Even if the entities are self-explanatory for this example, you can use `spacy.explain()` for a detailed description.

In [29]:
for ent in doc.ents:
    print(ent.text, ent.label_, spacy.explain(ent.label_))

Manchester United ORG Companies, agencies, institutions, etc.
Harry Kane PERSON People, including fictional
$90 million MONEY Monetary values, including unit


In [56]:
from spacy import displacy

displacy.render(sentence, style="ent")

We can also filter which entity to display:

In [57]:
sentence = nlp(u'Manchester United is looking to sign Harry Kane for $90 million. David demand 100 Million Dollars')
displacy.render(sentence, style='ent', jupyter=True)

In [59]:
filter = {'ents': ['ORG']}
displacy.render(sentence, style='ent', jupyter=True, options=filter)

## Preprocessing

To deal with text, it's often convenient to wrap all the preprocessing you want to do in a single function. Let's define one that:

- split into tokens
- remove stopwords
- remove punctuation
- make everything lowercase
- lemmatize it

In [16]:
def preprocessing(sentence):
    """
    params sentence: a str containing the sentence we want to preprocess
    return the tokens list
    """
    doc = nlp(sentence)
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_stop]
    return tokens

In [17]:
preprocessing("This is a sentence I'm going to preprocess")

['sentence', 'go', 'preprocess']

As you can see, removing "stopwords" can be brutal, cause you will be missing a lot of important words that helps for the meaning of the sentence. For example, we missed the word "I". For this reason, sometimes is better to specify a list of words that you think won't be meaningful for the task you're going to perform. A good way to find them, is by checking the most frequent words in the corpus you have.

Let's say you have this cell as text:

In [21]:
text = """As you can see, removing "stopwords" can be brutal, cause you will be missing a lot of important words that helps for the meaning of the sentence. For example, we missed the word "I". For this reason, sometimes is better to specify a list of words that you think won't be meaningful for the task you're going to perform. A good way to find them, is by checking the most frequent words in the corpus you have.

Let's say you have this cell as text:"""

In [24]:
from collections import Counter

counter = Counter()
counter.update(text.split(" "))

In [27]:
counter

Counter({'As': 1,
         'you': 5,
         'can': 2,
         'see,': 1,
         'removing': 1,
         '"stopwords"': 1,
         'be': 3,
         'brutal,': 1,
         'cause': 1,
         'will': 1,
         'missing': 1,
         'a': 2,
         'lot': 1,
         'of': 3,
         'important': 1,
         'words': 3,
         'that': 2,
         'helps': 1,
         'for': 2,
         'the': 6,
         'meaning': 1,
         'sentence.': 1,
         'For': 2,
         'example,': 1,
         'we': 1,
         'missed': 1,
         'word': 1,
         '"I".': 1,
         'this': 2,
         'reason,': 1,
         'sometimes': 1,
         'is': 2,
         'better': 1,
         'to': 3,
         'specify': 1,
         'list': 1,
         'think': 1,
         "won't": 1,
         'meaningful': 1,
         'task': 1,
         "you're": 1,
         'going': 1,
         'perform.': 1,
         'A': 1,
         'good': 1,
         'way': 1,
         'find': 1,
         'them,': 

By typing:

In [31]:
counter.most_common(10)

[('the', 6),
 ('you', 5),
 ('be', 3),
 ('of', 3),
 ('words', 3),
 ('to', 3),
 ('can', 2),
 ('a', 2),
 ('that', 2),
 ('for', 2)]

we get the 10 most common words. Let's say that I'm trying to get the gist of the text, then I could say that I get rid of "the" and "a" words. 

In that case, my preprocessing function becomes something like:


In [33]:
STOPWORDS = ["the", "a"]

def preprocessing(sentence):
    """
    params sentence: a str containing the sentence we want to preprocess
    return the tokens list
    """
    doc = nlp(sentence)
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.lemma_ in STOPWORDS]
    return tokens

where you see I replaced the control `not token.is_stop()` with `not token.lemma_ in STOPWORDS`.

As usual, be aware of the task you are going to perform is very important to improve the chances of an high accuracy instead of blindly applying a set of steps.